In [21]:
import pickle
import time
import myslack

from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from sklearn.model_selection import train_test_split

#### - train.csv

In [3]:
%%time
train = pickle.load(open("./data/my_train.pkl", "rb"))
print("test data shape: "+str(train.shape))

test data shape: (95674, 5166)
CPU times: user 154 ms, sys: 467 ms, total: 622 ms
Wall time: 651 ms


In [4]:
train.shape

(95674, 5166)

#### - test.csv

In [5]:
# %%time
test = pickle.load(open("./data/my_test.pkl", "rb"))
print("test data shape: "+str(test.shape))

test data shape: (95674, 5165)


In [6]:
test.shape

(95674, 5165)

---

# KNN - distance

### Train_X, Train_y

In [35]:
X = train.drop(columns='TripType')
y = train['TripType']

In [36]:
len(pd.Series(y).unique())

38

In [37]:
%time knn_model = KNN(n_neighbors=38, weights='distance', n_jobs=6).fit(X,y)

CPU times: user 1min 25s, sys: 5.08 s, total: 1min 30s
Wall time: 1min 31s


In [38]:
myslack.send_slack('knn_model modeling finish!')

<Response [200]>


### Confusion Matrix

In [39]:
%time confusion_matrix(y, knn_model.predict(X))

CPU times: user 1h 16min 37s, sys: 28.9 s, total: 1h 17min 6s
Wall time: 13min 30s


array([[3597,    0,    0, ...,    0,    0,   46],
       [   0,  346,    0, ...,    0,    0,    0],
       [   0,    0, 4593, ...,    0,    0,    0],
       ...,
       [   0,    0,    0, ...,  872,    0,    0],
       [   0,    0,    0, ...,    0, 1187,    0],
       [ 251,    2,    2, ...,    0,    0, 8125]])

In [40]:
myslack.send_slack('knn_model prediction finish!')

<Response [200]>


### Train Test Split

In [41]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

X_train.shape[0], X_test.shape[0], len(y_train), len(y_test)

(86106, 9568, 86106, 9568)

In [42]:
%time y_pred = knn_model.predict(X_test)

CPU times: user 7min 33s, sys: 2.07 s, total: 7min 35s
Wall time: 1min 18s


In [43]:
myslack.send_slack('knn_model prediction finish!')

<Response [200]>


In [44]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          3       0.94      0.98      0.96       406
          4       1.00      1.00      1.00        28
          5       1.00      1.00      1.00       436
          6       1.00      1.00      1.00       114
          7       1.00      1.00      1.00       589
          8       1.00      1.00      1.00      1267
          9       0.99      0.99      0.99       926
         12       1.00      1.00      1.00        29
         15       0.99      0.99      0.99        98
         18       1.00      1.00      1.00        53
         19       0.98      1.00      0.99        43
         20       1.00      1.00      1.00        55
         21       1.00      1.00      1.00        58
         22       0.98      0.99      0.98        85
         23       1.00      1.00      1.00         9
         24       1.00      1.00      1.00       244
         25       1.00      1.00      1.00       386
         26       1.00      1.00      1.00   

### prediction

In [45]:
%time result = knn_model.predict_proba(test)

CPU times: user 1h 28min 44s, sys: 33.2 s, total: 1h 29min 18s
Wall time: 17min 29s


In [46]:
myslack.send_slack('knn_model prediction finish!')

<Response [200]>


In [47]:
result

array([[0.        , 0.02599796, 0.02431887, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.05167047, ..., 0.        , 0.        ,
        0.05671425],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.0201755 , ..., 0.        , 0.        ,
        0.        ],
       [0.02532469, 0.        , 0.02150639, ..., 0.        , 0.        ,
        0.        ]])

### Make Submission CSV file

In [48]:
samsub = pd.read_csv('./data/sample_submission.csv')
subform_df_columns = samsub.columns[1:]
result_df = pd.DataFrame(result)
result_df.columns = subform_df_columns
subform_df = pd.concat([test.reset_index()['VisitNumber'],result_df],axis=1)
subform_df.set_index('VisitNumber',inplace=True)
subform_df.tail()

,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,TripType_15,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
VisitNumber,,,,,,,,,,,,,,,,,,,,,
191338,0.000000,0.0,0.054190,0.000000,0.100681,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.064664,0.054343,0.208762,0.0,0.000000,0.075091,0.0,0.0,0.0
191339,0.000000,0.0,0.027331,0.000000,0.053736,0.0,0.0,0.0,0.0,0.0,...,0.028363,0.032339,0.027331,0.189621,0.0,0.000000,0.072711,0.0,0.0,0.0
191340,0.000000,0.0,0.000000,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0
191341,0.000000,0.0,0.020176,0.000000,0.103789,0.0,0.0,0.0,0.0,0.0,...,0.083293,0.020679,0.061131,0.469382,0.0,0.022388,0.020226,0.0,0.0,0.0
191348,0.025325,0.0,0.021506,0.050649,0.025325,0.0,0.0,0.0,0.0,0.0,...,0.130871,0.065986,0.000000,0.144979,0.0,0.000000,0.030086,0.0,0.0,0.0


In [49]:
subform_df.to_csv('./KNN/knn_distance_model.csv')

In [50]:
myslack.send_slack('Making knn_distance_model to CSV finish!!')

<Response [200]>


---

## 모델 저장

In [51]:
from sklearn.externals import joblib

joblib.dump(knn_model, './KNN/knn_distance_model.pkl')
knn_uniform_model = joblib.load('./KNN/knn_distance_model.pkl')

In [52]:
myslack.send_slack('Making knn_distance_model to model pkl finish!!')

<Response [200]>


---

---

# KNN - uniform

### Train_X, Train_y

In [35]:
X = train.drop(columns='TripType')
y = train['TripType']

In [36]:
len(pd.Series(y).unique())

38

In [ ]:
%time knn_model = KNN(n_neighbors=38, weights='uniform', n_jobs=6).fit(X,y)

In [ ]:
myslack.send_slack('knn_model modeling finish!')

In [12]:
%time knn_model = KNN(n_neighbors=38, weights='uniform', n_jobs=6).fit(X,y)

CPU times: user 1min 22s, sys: 4.71 s, total: 1min 27s
Wall time: 1min 28s


In [15]:
myslack.send_slack('knn_model modeling finish!')

<Response [200]>


### Confusion Matrix

In [18]:
%time confusion_matrix(y, knn_model.predict(X))

CPU times: user 1h 17min 3s, sys: 30.7 s, total: 1h 17min 33s
Wall time: 13min 38s


array([[3440,    0,   13, ...,    0,    0,   47],
       [   1,   24,  189, ...,    0,    0,    0],
       [  33,    0, 2971, ...,    1,    0,    3],
       ...,
       [   0,    0,   11, ...,    5,    0,    0],
       [   0,    0,    0, ...,    0,    0,    0],
       [ 406,    0,   78, ...,    0,    0, 6726]])

In [19]:
myslack.send_slack('knn_model prediction finish!')

<Response [200]>


### Train Test Split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

X_train.shape[0], X_test.shape[0], len(y_train), len(y_test)

(86106, 9568, 86106, 9568)

In [23]:
%time y_pred = knn_model.predict(X_test)

CPU times: user 7min 40s, sys: 1.7 s, total: 7min 42s
Wall time: 1min 20s


In [24]:
myslack.send_slack('knn_model prediction finish!')

<Response [200]>


In [25]:
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          3       0.75      0.95      0.84       401
          4       1.00      0.07      0.12        30
          5       0.73      0.67      0.70       438
          6       0.81      0.47      0.59       120
          7       0.58      0.40      0.48       587
          8       0.61      0.92      0.73      1229
          9       0.71      0.64      0.68       970
         12       0.00      0.00      0.00        26
         14       0.00      0.00      0.00         1
         15       0.60      0.03      0.06        95
         18       1.00      0.02      0.04        47
         19       0.67      0.11      0.19        36
         20       0.71      0.07      0.12        73
         21       0.50      0.06      0.10        69
         22       0.77      0.12      0.20        86
         23       0.00      0.00      0.00        18
         24       0.39      0.06      0.11       277
         25       0.32      0.30      0.31   

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### prediction

In [27]:
%time result = knn_model.predict_proba(test)

CPU times: user 1h 16min 4s, sys: 23.2 s, total: 1h 16min 27s
Wall time: 13min 17s


In [28]:
myslack.send_slack('knn_model prediction finish!')

<Response [200]>


In [29]:
result

array([[0.        , 0.02631579, 0.02631579, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.05263158, ..., 0.        , 0.        ,
        0.05263158],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.05263158],
       [0.        , 0.        , 0.02631579, ..., 0.        , 0.        ,
        0.        ],
       [0.02631579, 0.        , 0.02631579, ..., 0.        , 0.        ,
        0.        ]])

### Make Submission CSV file

In [30]:
samsub = pd.read_csv('./data/sample_submission.csv')
subform_df_columns = samsub.columns[1:]
result_df = pd.DataFrame(result)
result_df.columns = subform_df_columns
subform_df = pd.concat([test.reset_index()['VisitNumber'],result_df],axis=1)
subform_df.set_index('VisitNumber',inplace=True)
subform_df.tail()

,TripType_3,TripType_4,TripType_5,TripType_6,TripType_7,TripType_8,TripType_9,TripType_12,TripType_14,TripType_15,...,TripType_36,TripType_37,TripType_38,TripType_39,TripType_40,TripType_41,TripType_42,TripType_43,TripType_44,TripType_999
VisitNumber,,,,,,,,,,,,,,,,,,,,,
191338,0.000000,0.0,0.052632,0.000000,0.105263,0.000000,0.000000,0.0,0.0,0.000000,...,0.000000,0.052632,0.052632,0.210526,0.0,0.000000,0.078947,0.0,0.0,0.000000
191339,0.000000,0.0,0.026316,0.000000,0.052632,0.000000,0.000000,0.0,0.0,0.000000,...,0.026316,0.026316,0.026316,0.184211,0.0,0.000000,0.078947,0.0,0.0,0.000000
191340,0.000000,0.0,0.000000,0.000000,0.078947,0.605263,0.210526,0.0,0.0,0.026316,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.052632
191341,0.000000,0.0,0.026316,0.000000,0.078947,0.000000,0.000000,0.0,0.0,0.000000,...,0.078947,0.026316,0.078947,0.473684,0.0,0.026316,0.026316,0.0,0.0,0.000000
191348,0.026316,0.0,0.026316,0.052632,0.026316,0.000000,0.000000,0.0,0.0,0.000000,...,0.131579,0.052632,0.000000,0.157895,0.0,0.000000,0.026316,0.0,0.0,0.000000


In [33]:
subform_df.to_csv('./KNN/knn_uniform_model.csv')

In [34]:
myslack.send_slack('Making knn_uniform_model to CSV finish!!')

<Response [200]>


---

## 모델 저장

In [31]:
from sklearn.externals import joblib

joblib.dump(knn_model, './KNN/knn_uniform_model.pkl')
knn_uniform_model = joblib.load('./KNN/knn_uniform_model.pkl')

In [32]:
myslack.send_slack('Making knn_uniform_model to model pkl finish!!')

<Response [200]>
